In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240706'

In [3]:
import os

list_file = sorted(list(os.walk(f'./{param_date}/11.part_2'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['part_1-fragment_1.xlsx',
 'part_10-fragment_10.xlsx',
 'part_11-fragment_11.xlsx',
 'part_12-fragment_12.xlsx',
 'part_13-fragment_13.xlsx',
 'part_14-fragment_14.xlsx',
 'part_15-fragment_15.xlsx',
 'part_16-fragment_16.xlsx',
 'part_17-fragment_17.xlsx',
 'part_18-fragment_18.xlsx',
 'part_19-fragment_19.xlsx',
 'part_2-fragment_2.xlsx',
 'part_20-fragment_20.xlsx',
 'part_21-fragment_21.xlsx',
 'part_22-fragment_22.xlsx',
 'part_23-fragment_23.xlsx',
 'part_24-fragment_24.xlsx',
 'part_25-fragment_25.xlsx',
 'part_26-fragment_26.xlsx',
 'part_27-fragment_27.xlsx',
 'part_28-fragment_28.xlsx',
 'part_29-fragment_29.xlsx',
 'part_3-fragment_3.xlsx',
 'part_30-fragment_30.xlsx',
 'part_31-fragment_31.xlsx',
 'part_32-fragment_32.xlsx',
 'part_33-fragment_33.xlsx',
 'part_4-fragment_4.xlsx',
 'part_5-fragment_5.xlsx',
 'part_6-fragment_6.xlsx',
 'part_7-fragment_7.xlsx',
 'part_8-fragment_8.xlsx',
 'part_9-fragment_9.xlsx']

In [4]:
for file in list_file:

    # = = = = = = = = = = = = = = =

    import pandas as pd

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./{param_date}/11.part_2/{file}',
                           header=0,
                           dtype=str).fillna('')

    input_ = input_[input_['Vehicle Url'] != ''].reset_index(drop=True)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =

    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                import requests

                import sys
                sys.path.append('../00.Tools')
                from crawler_configuration import get_header, get_proxy

                from bs4 import BeautifulSoup
                from lxml import etree

                import json

                df_temp, i, next_ = pd.DataFrame(), 0, ['Lennon']
                while next_:
                    try:
                        resp = requests.get(f'''{input_.loc[a, 'Vehicle Url']}&start={i*50}&num=50''',
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10)).text

                        # = = = = = = = = = = = = = = =

                        soup = BeautifulSoup(resp.strip(), 'lxml')
                        html = etree.HTML(str(soup))

                        # = = = = = = = = = = = = = = =

                        if len(html.xpath('//div[@class="detail-app-pager"]')) == 0:
                            continue

                        # = = = = = = = = = = = = = = =

                        next_ = html.xpath('//a[@id="pagingBottom_nextButton"]')
                        if next_ and next_[0].xpath('./@href')[0].strip() == 'Search/#':
                            break

                        # = = = = = = = = = = = = = = =

                        list_th = [text.strip() for text in html.xpath('//th[@scope="col"]/text()')]

                        # = = = = = = = = = = = = = = =

                        list_tr = html.xpath('//tr[@class="detail-app-row"]')

                        # = = = = = = = = = = = = = = =

                        df_page = pd.DataFrame({'Part Number': input_.loc[a, 'Part Number'],
                                                'Vehicle Url': input_.loc[a, 'Vehicle Url'],
                                                'Vehicle Page': i+1,
                                                'Row': [j+1 for j in range(len(list_tr))]})

                        for j, tr in zip(range(len(list_tr)), list_tr):
                            for k, th in zip(range(len(list_th)), list_th):
                                df_page.loc[j, th] = tr.xpath(f'./td[{k+1}]/text()')[0].strip()

                        df_temp = pd.concat([df_temp, df_page], ignore_index=True).fillna('')

                        # = = = = = = = = = = = = = = =

                        i += 1

                    except:
                        continue

                # = = = = = = = = = = = = = = =

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

            # = = = = = = = = = = = = = = =

            except:
                pass

            # = = = = = = = = = = = = = = =

            if crawler_status == 'error':
                df_temp = pd.DataFrame([{'Part Number': input_.loc[a, 'Part Number'],
                                         'Vehicle Url': input_.loc[a, 'Vehicle Url']}])

                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'SKU No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(20):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Part Number', 'Vehicle Url', 'Vehicle Page', 'Row'],
                                                    ascending=[True, True, True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./{param_date}/12.vehicle_1/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    iif not output_error.empty:
        output_error = output_error.sort_values(by=['Part Number', 'Vehicle Url'],
                                                ascending=[True, True],
                                                ignore_index=True)
        output_error.to_excel(f'''./{param_date}/0.error/{file.removesuffix('.xlsx')}-vehicle_1_error.xlsx''', index=False)
        print('爬虫存在error')
        print()
    print('Done ~')

[状态：ok] - 100.00% - part_9-fragment_9 > 25669.926-897
[剩余数量：1000] - [当前时间：05:32:17]

[状态：ok] - 100.00% - part_9-fragment_9 > 25671.973-150
[剩余数量：999] - [当前时间：05:32:17]

[状态：ok] - 100.00% - part_9-fragment_9 > 25675.624-559
[剩余数量：998] - [当前时间：05:32:18]

[状态：ok] - 100.00% - part_9-fragment_9 > 25656.674-9037
[剩余数量：997] - [当前时间：05:32:18]

[状态：ok] - 100.00% - part_9-fragment_9 > 25674.601-231
[剩余数量：996] - [当前时间：05:32:18]

[状态：ok] - 100.00% - part_9-fragment_9 > 25638.970-026
[剩余数量：995] - [当前时间：05:32:19]

[状态：ok] - 100.00% - part_9-fragment_9 > 25682.674-312
[剩余数量：994] - [当前时间：05:32:19]

[状态：ok] - 100.00% - part_9-fragment_9 > 25677.97031.1
[剩余数量：993] - [当前时间：05:32:19]

[状态：ok] - 100.00% - part_9-fragment_9 > 25604.926-908
[剩余数量：992] - [当前时间：05:32:20]

[状态：ok] - 100.00% - part_9-fragment_9 > 25678.625-829
[剩余数量：991] - [当前时间：05:32:20]

[状态：ok] - 100.00% - part_9-fragment_9 > 25683.698-228
[剩余数量：990] - [当前时间：05:32:20]

[状态：ok] - 100.00% - part_9-fragment_9 > 25685.698-303
[剩余数量：989] - [当前时间：0